In [ ]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType, FloatType, BooleanType, DateType
import datetime

def profile_and_cast_dataframe(df):
    """
    Profiles a PySpark DataFrame with string columns and attempts to infer and cast columns to appropriate data types.
    
    Parameters:
        df (DataFrame): Input PySpark DataFrame with all string columns.
    
    Returns:
        DataFrame: A new DataFrame with columns cast to inferred data types.
    """
    def infer_data_type(value):
        if value is None:
            return None
        try:
            int(value)
            return IntegerType()
        except ValueError:
            pass
        try:
            float(value)
            return FloatType()
        except ValueError:
            pass
        if value.lower() in ['true', 'false']:
            return BooleanType()
        try:
            datetime.datetime.strptime(value, '%Y-%m-%d')
            return DateType()
        except ValueError:
            pass
        return None

    inferred_types = {}
    for column in df.columns:
        sample_values = df.select(column).distinct().limit(100).rdd.flatMap(lambda x: x).collect()
        types = set(filter(None, (infer_data_type(value) for value in sample_values)))
        
        if len(types) == 1:
            inferred_types[column] = types.pop()
        else:
            inferred_types[column] = None

    for column, data_type in inferred_types.items():
        if data_type == IntegerType():
            df = df.withColumn(column, when(col(column).rlike('^-?\\d+$'), col(column).cast(IntegerType())))
        elif data_type == FloatType():
            df = df.withColumn(column, when(col(column).rlike('^-?\\d*\\.\\d+$'), col(column).cast(FloatType())))
        elif data_type == BooleanType():
            df = df.withColumn(column, when(col(column).rlike('^(true|false)$'), col(column).cast(BooleanType())))
        elif data_type == DateType():
            df = df.withColumn(column, when(col(column).rlike('^\\d{4}-\\d{2}-\\d{2}$'), col(column).cast(DateType())))

    return df